In [1]:
%matplotlib inline

In [35]:
from gahyparopt.widgets import *
from gahyparopt.GAUtilities import *

In [3]:
mnist = load_data()

In [18]:
import glob

In [23]:
players = [fn.split(".")[0] for fn in glob.glob('*.json')]

In [24]:
players

['Frederic',
 'Ellen',
 'Paul',
 'Norma',
 'Dave',
 'Michael',
 'AgathElisa',
 'Loukas',
 'Pauline',
 'cmf',
 'Carsten',
 'Andy',
 'Jana',
 'Corinna']

In [25]:
games = dict([(p, game_instance(
            ga=GADriver(
            layer_counts=HIDDEN_LAYER_COUNT,
            no_neurons=HIDDEN_LAYER_NEURONS,
            rates=HIDDEN_LAYER_RATE,
            activations=HIDDEN_LAYER_ACTIVATIONS,
            layer_types=HIDDEN_LAYER_TYPE,
            optimizers=MODEL_OPTIMIZER,
            population_size=1,
            best_candidates_count=BEST_CANDIDATES_COUNT,
            random_candidates_count=RANDOM_CANDIDATES_COUNT,
            optimizer_mutation_probability=OPTIMIZER_MUTATION_PROBABILITY,
            layer_mutation_probability=HIDDEN_LAYER_MUTATION_PROBABILITY,
        ))
    ) for p in players])

In [28]:
%%capture
# Init all players with data and model.
for player, game in games.items():
    game.name = player
    game.data = mnist
    game.individuum = create_start_individuum(game.ga)[0]
    
    
    
    
    

In [26]:
generation = 2

In [37]:
# Training
clear_keras_session()
for player, game in games.items():
    print("************ {} ******************".format(player))
    sync_remote_to_local(game.name)
    if generation > 0:
        games[player].individuum = read_chromosome(game.name)
    
    try:
        games[player].history = train_individuum(game.ga, game.data, game.individuum)
        
    except:
        games[player].history = None
    
    if games[player].history is not None:
        try:
                loss, accuracy = evaluate_model(game.individuum.ml_model, game.data['x_val'], y=game.data['y_val'])
        except:
            accuracy = 0.0
            loss = 100.0

    else:
        accuracy = 0.0
        loss = 100.0

    games[player].individuum.loss = loss
    games[player].individuum.accuracy = accuracy
    
    write_chromosome(game.name, game.individuum)
    sync_local_to_remote(game.name)
    
generation += 1 
    
    

************ Frederic ******************
Epoch 1/10
10/10 [==============================] - 1s 145ms/step - loss: 1.8243 - accuracy: 0.4558 - val_loss: 1.3840 - val_accuracy: 0.7229
Epoch 2/10
10/10 [==============================] - 1s 143ms/step - loss: 1.2519 - accuracy: 0.7290 - val_loss: 1.0525 - val_accuracy: 0.8014
Epoch 3/10
10/10 [==============================] - 1s 136ms/step - loss: 0.9997 - accuracy: 0.7877 - val_loss: 0.8627 - val_accuracy: 0.8314
Epoch 4/10
10/10 [==============================] - 1s 135ms/step - loss: 0.8470 - accuracy: 0.8155 - val_loss: 0.7302 - val_accuracy: 0.8511
Epoch 5/10
10/10 [==============================] - 1s 137ms/step - loss: 0.7360 - accuracy: 0.8362 - val_loss: 0.6411 - val_accuracy: 0.8624
Epoch 6/10
10/10 [==============================] - 1s 145ms/step - loss: 0.6537 - accuracy: 0.8481 - val_loss: 0.5679 - val_accuracy: 0.8770
Epoch 7/10
10/10 [==============================] - 1s 137ms/step - loss: 0.5909 - accuracy: 0.8613 - val_l

# Danger zone!!!

In [17]:
delete_local()
delete_remote()